In [1]:
from scipy.stats import truncnorm
import pandas as pd
import numpy as np
import itertools
import datetime
import tqdm
import sys
import os

def flatten_list(list_array):
    return list(itertools.chain(*list_array))

sys.path.insert(0, "../")
sys.path.insert(0,"../pompjax/pompjax/")

from global_config import config

results_dir           = config.get_property('results_dir')
results2_dir           = config.get_property('results2_dir')

data_dir              = config.get_property('data_dir')
paper_dir             = config.get_property('paper_dir')
data_db_dir           = config.get_property('data_db_dir')
feb_hosp_records_path = os.path.join(data_db_dir, 'long_files_8_25_2021')
path_to_save          = os.path.join(results_dir, "real_testing", "community")

COLOR_LIST1           = ["#F8AFA8", "#FDDDA0", "#F5CDB4", "#74A089"]

# pompjax imports
from utils_local.misc import amro2title, amro2cute
import matplotlib.pyplot as plt

from eval.calibration import calibration
from utils import create_df_response


In [2]:
from utils_data_metapop import create_population_data, create_time_transfers

path_to_ward_counts = os.path.join(data_db_dir, "long_files_8_25_2021", "counts_ward.csv" )
path_to_ward_transf = os.path.join(data_db_dir, "long_files_8_25_2021", "transfers_ward.csv" )

A_df, D_df, H_df, tests_df, Hmean_df = create_population_data(path_to_ward_counts)

num_wards  = len(Hmean_df)
ward_names = list(Hmean_df.index)
M_df       = create_time_transfers(path_to_ward_transf, num_wards=num_wards, ward_names=ward_names)

selected_buildings = ['Allen Hospital-Allen', 'Harkness Pavilion-Columbia', 'Milstein Hospital-Columbia', 'Mschony-Chony', 'Presbyterian Hospital-Columbia']
building2id        = {selected_buildings[i]: i for i in range(len(selected_buildings))}

def building2observation(building):
    if building in selected_buildings:
        return building2id[building]
    else:
        return 5

ward_names_df                = pd.DataFrame(ward_names, columns=["ward"])
ward_names_df["building"]    = ward_names_df["ward"].apply(lambda x: "-".join(x.split("-")[1:]))
ward_names_df["buidling_id"] = ward_names_df["building"].apply(lambda x: building2observation(x) )
ward_names_df["ward_id"]     = ward_names_df.apply(lambda x: np.where(ward_names_df.ward == x.ward)[0][0], axis=1)
wardid2buildingid            = {row.ward_id: row.buidling_id for i, row in ward_names_df.iterrows()}


from models import process_metapop, observe_metapop_cluster, init_metapop, simulate_metapop, simulate_metapop_observations
from utils_local.misc import amro2cute

delta = 1/120  # decolonization rate
A     = A_df.to_numpy()
D     = D_df.to_numpy()
H     = H_df.to_numpy()
M     = M_df
tests = tests_df.to_numpy()


In [3]:
#### create scenarios ####
amro_search  = ['ESCHERICHIA COLI', 'KLEBSIELLA PNEUMONIAE',  'PSEUDOMONAS AERUGINOSA',
            'METHICILLIN-SUSCEPTIBLE STAPHYLOCOCCUS AUREUS', 'METHICILLIN-RESISTANT STAPHYLOCOCCUS AUREUS',
            "STAPHYLOCOCCUS EPIDERMIDIS", 'ENTEROCOCCUS FAECALIS', 'ENTEROCOCCUS FAECIUM']

path_to_scenarios         = os.path.join(results2_dir, "synthetic_inferences", "metapopulation")


## calibration not adjusting the state space

In [4]:
dates_simulation = pd.date_range(start=pd.to_datetime("2020-02-01"), end=pd.to_datetime("2021-02-28"), freq="D")
num_pop          = num_wards

model_settings = {
                "n"              : 3 * num_pop,           # number of state variables / dimension of the state space
                "T"              : len(dates_simulation),  # time to run
                "num_pop"        : num_pop,
                "dates"          : dates_simulation,
                "num_build"      : len(np.unique(list(wardid2buildingid.values()))),
                "k"              : len(np.unique(list(wardid2buildingid.values()))) # observing at the building aggregation
                }

In [5]:
from utils_data_metapop import empirical_prevalence
from tqdm import tqdm

for idx_amro, amro in enumerate(amro_search):
    print("Running IF-EAKF for amro: ", amro2title(amro))
    path_to_save = os.path.join(results2_dir, "synthetic_inferences", "metapopulation", f"{amro2cute(amro)}")
    scenarios_df = pd.read_csv(os.path.join(path_to_save, "scenarios.csv"))

    gamma        = empirical_prevalence(amro, path_to_prev="../data/amro_prevalence.csv")

    calibration_df = pd.DataFrame()
    for idx_row, row in scenarios_df.iterrows():
        #idx_row, row = next(scenarios_df.iterrows())
        inference = np.load(os.path.join(path_to_save, "no_adjust_state_space",
                                        f"scenario{idx_row+1}", f"{str(0).zfill(3)}posterior.npz"))

        θmle      = inference["mle"]
        θpost     = inference["posterior"]
        x_sim     = inference["state_space"]
        y_sim     = inference["observations"]
        θtruth    = inference["teta_truth"]
        idx_infer = inference["idx_infer"]
        Nif       = θpost.shape[-1]

        model_settings["m"] = θpost.shape[1]

        f0  = lambda θ:  init_metapop(N0               = H[:, 0],
                                        c0             = gamma,
                                        model_settings = model_settings)

        f  = lambda t, x, θ: process_metapop(t, x,
                                                gamma = gamma * np.ones(model_settings["m"]),
                                                beta  = θ[1, :],
                                                delta = delta,
                                                Nmean = np.expand_dims(Hmean_df, -1),
                                                N     = H[:, [t]],
                                                A     = A[:, [t]],
                                                D     = D[:, [t]],
                                                M     = M[:, :, t])

        g = lambda t, x, θ: observe_metapop_cluster(t, x,
                                                    rho            = θ[0, :],
                                                    N              = H[:, [t]],
                                                    num_tests      = tests[:, [t]],
                                                    model_settings = model_settings,
                                                    ward2cluster   = wardid2buildingid)

        observation = np.sum(y_sim[:, :, idx_infer].transpose(1, 0), axis=0, keepdims=True)

        θpost = np.mean(θpost, -2)

        for if_iter in tqdm(range(Nif)):
            θsim  = θpost[:, :, if_iter]
            x_sim, obs_sim     = simulate_metapop(process_model = f,
                                            observational_model = g,
                                            init_state          = f0,
                                            θsim                = θsim,
                                            model_settings      = model_settings)

            obs_sim           = np.sum(obs_sim.transpose(1, 0, 2) , axis=0, keepdims=True)
            cal_df            = calibration(obs_sim, observation, observation_index=0, dates=model_settings["dates"])
            cal_df["if_iter"] = if_iter
            cal_df["score"]   = np.mean(np.abs(cal_df.quantiles.values - cal_df.proportion_inside.values))
            calibration_df    = pd.concat([calibration_df, cal_df])
        calibration_df["scenario"] = f"scenario{idx_row+1}"

    calibration_df["amro"] = amro
    calibration_df.to_csv(os.path.join(path_to_save, "no_adjust_state_space", "calibration.csv"), index=False)


Running IF-EAKF for amro:  E. coli


100%|██████████| 30/30 [08:05<00:00, 16.18s/it]


Running IF-EAKF for amro:  K. pneumoniae


100%|██████████| 30/30 [07:38<00:00, 15.29s/it]


Running IF-EAKF for amro:  P. aeruginosa


100%|██████████| 30/30 [08:18<00:00, 16.60s/it]


Running IF-EAKF for amro:  MSSA


 53%|█████▎    | 16/30 [04:33<03:48, 16.33s/it]

In [ ]:
import seaborn as sns

fig, ax = plt.subplots(1, 1, figsize=(7, 7))
#ax.plot(cal_df.quantiles, cal_df.proportion_inside, color="red", marker="o")
sns.lineplot(ax=ax, x="quantiles", y="proportion_inside", data=calibration_df, hue="if_iter", legend=True, palette="mako", marker="o")
ax.plot(np.linspace(0,1), np.linspace(0,1), ls="--", color="k")
ax.legend(title="IF iteration")
ax.set_xlabel("Predictive CI")
ax.set_ylabel("Observed CI")


In [ ]:
cal_time_df = calibration_df[["if_iter", "score"]].groupby("if_iter").mean()

fig, ax = plt.subplots(1, 1, figsize=(7.2, 5))
ax.plot(cal_time_df.index, cal_time_df.score, marker="o", lw=2, c="k")
ax.set_xlabel("IF iteration")
ax.set_ylabel(r"Calibration score: $\mathbb{E}\left\{|CI - PI|\right\}$")
ax.set_title(f"{amro2title(amro)} calibration")